In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

pd.set_option("display.max_columns", 50)

In [2]:
img_path = os.path.exists("../img/")
if img_path == False:
    os.mkdir("../img/")
else:
    print("path already exists")

path already exists


In [3]:
merged = pd.read_csv("../nbs/data/merged.csv")

merged.head()

/var/folders/xf/_6m_tb052fn5p84tlt773rqr0000gn/T/ipykernel_1767/3301346813.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  merged = pd.read_csv("../nbs/data/merged.csv")


,sku,national_inv,lead_time,in_transit_qty,forecast_3_month,forecast_6_month,forecast_9_month,sales_1_month,sales_3_month,sales_6_month,sales_9_month,min_bank,potential_issue,pieces_past_due,perf_6_month_avg,perf_12_month_avg,local_bo_qty,deck_risk,oe_constraint,ppap_risk,stop_auto_buy,rev_stop,went_on_backorder
0,1026827,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,No,0.0,-99.00,-99.00,0.0,No,No,No,Yes,No,No
1,1043384,2.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,No,0.0,0.99,0.99,0.0,No,No,No,Yes,No,No
2,1043696,2.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,No,0.0,-99.00,-99.00,0.0,Yes,No,No,Yes,No,No
3,1043852,7.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,No,0.0,0.10,0.13,0.0,No,No,No,Yes,No,No
4,1044048,8.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,2.0,No,0.0,-99.00,-99.00,0.0,Yes,No,No,Yes,No,No


## CHECK FOR MISSING VALUES

In [4]:
merged.isnull().sum()
missing_values_df = pd.DataFrame(columns=['MissingVal%'], index=merged.columns)
for col in merged.columns:
    missing_values_df.loc[col, 'MissingVal%'] = np.ceil((merged[col].isnull().sum()/merged.shape[0]) * 100)

missing_values_df

,MissingVal%
sku,0.0
national_inv,1.0
lead_time,6.0
in_transit_qty,1.0
forecast_3_month,1.0
forecast_6_month,1.0
forecast_9_month,1.0
sales_1_month,1.0
sales_3_month,1.0
sales_6_month,1.0


## CHECKING FOR DATA TYPES

In [5]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1929937 entries, 0 to 1929936
Data columns (total 23 columns):
 #   Column             Dtype  
---  ------             -----  
 0   sku                object 
 1   national_inv       float64
 2   lead_time          float64
 3   in_transit_qty     float64
 4   forecast_3_month   float64
 5   forecast_6_month   float64
 6   forecast_9_month   float64
 7   sales_1_month      float64
 8   sales_3_month      float64
 9   sales_6_month      float64
 10  sales_9_month      float64
 11  min_bank           float64
 12  potential_issue    object 
 13  pieces_past_due    float64
 14  perf_6_month_avg   float64
 15  perf_12_month_avg  float64
 16  local_bo_qty       float64
 17  deck_risk          object 
 18  oe_constraint      object 
 19  ppap_risk          object 
 20  stop_auto_buy      object 
 21  rev_stop           object 
 22  went_on_backorder  object 
dtypes: float64(15), object(8)
memory usage: 338.7+ MB


In [8]:
merged['oe_constraint'].mode()[0]

'No'

In [ ]:
#remove the two rows of all NA's
merged=merged[pd.notnull(merged['national_inv'])]

In [ ]:
quantvars = merged.select_dtypes(include=['float64']).columns.to_list()

cat_cols = merged.select_dtypes(include='object').columns.to_list()
cat_cols.remove('sku')
for category in cat_cols:
    print(f'👉 {category} has {merged[category].nunique()} values')

catpred = cat_cols.remove('went_on_backorder')

### Encoding Category columns

In [ ]:
for category in cat_cols:
    merged[category] = merged[category].astype('category').cat.codes

In [ ]:
merged['perf_12_month_avg'] = merged['perf_12_month_avg'].replace(-99, np.NaN)
merged['perf_6_month_avg'] = merged['perf_6_month_avg'].replace(-99, np.NaN)

merged.head(5)

### DESCRIPTIVE STATISTICS AND PLOT

In [ ]:
merged[quantvars].describe().transpose()

In [ ]:
fig, axs= plt.subplots(3, 3, figsize = (15, 6))
sns.scatterplot(x=merged.forecast_3_month, y=merged.sales_3_month, ax=axs[0, 0])
sns.scatterplot(x=merged.forecast_6_month, y=merged.sales_6_month, ax=axs[0, 1])
sns.scatterplot(x=merged.forecast_9_month, y=merged.sales_9_month, ax=axs[0, 2])
sns.scatterplot(x=merged.national_inv, y=merged.sales_1_month, ax=axs[1, 0])
sns.scatterplot(x=merged.national_inv, y=merged.sales_3_month, ax=axs[1, 1])
sns.scatterplot(x=merged.national_inv, y=merged.sales_6_month, ax=axs[1, 2])
sns.scatterplot(x=merged.national_inv, y=merged.sales_9_month, ax=axs[2, 0])
sns.scatterplot(x=merged.in_transit_qty, y=merged.national_inv, ax=axs[2, 1])
sns.scatterplot(x=merged.in_transit_qty, y=merged.lead_time, ax=axs[2, 2])

plt.savefig("../img/Scatter_plot_for_sales.png")

In [ ]:
n_columns = 3
n_rows = 5
_, axs = plt.subplots(n_rows, n_columns, figsize=(8 * n_columns, 5 * n_rows))
for i, c in enumerate(quantvars):
    sns.boxplot(y= c, data= merged, ax= axs[i // n_columns, i % n_columns])

plt.title("Boxplot with Quantitative Columns")
plt.tight_layout()
plt.show()

plt.savefig("../img/Boxplot_quant_cols.png")

In [ ]:
catpred = cat_cols
catpred

In [ ]:
fig, axs = plt.subplots(3, 2, figsize = (10, 5))
# plt.subplots_adjust(hspace=0.1, wspace=0.4)
pd.crosstab(merged.stop_auto_buy, merged.went_on_backorder).plot(kind='bar', ax=axs[0, 0], title="Stop Auto Buy")
pd.crosstab(merged.potential_issue, merged.went_on_backorder).plot(kind='bar', ax=axs[0, 1], title="Potential Issue")
pd.crosstab(merged.oe_constraint, merged.went_on_backorder).plot(kind='bar', ax=axs[1, 0], title="OE Constraint")
pd.crosstab(merged.rev_stop, merged.went_on_backorder).plot(kind='bar', ax=axs[1, 1], title="Rev Stop")
pd.crosstab(merged.ppap_risk, merged.went_on_backorder).plot(kind='bar', ax=axs[2, 0], title="PPAP Risk")
pd.crosstab(merged.deck_risk, merged.went_on_backorder).plot(kind='bar', ax=axs[2, 1], title="Deck Risk")
plt.savefig("../img/Categories_with_targets.png")

### Correlation Plot

In [ ]:
corr_col = merged.columns.tolist()
corr_col.remove('sku')
corr_col.remove('went_on_backorder')
print(corr_col)

In [ ]:
#correlation matrix

corrmat = merged[corr_col].corr()
f, ax = plt.subplots(figsize = (12, 9))
sns.heatmap(corrmat, vmax=0.6, square=True, annot=True)
plt.savefig("../img/correlation.png")

### Missing Data and Treatment

In [ ]:
total = merged.isnull().sum().sort_values(ascending=False)
percent = (merged.isnull().sum()/ merged.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data


In [ ]:
missing_ele= missing_data[missing_data['Total'] > 1].index.to_list()
for ele in missing_ele:
    merged[ele]= merged[ele].replace(np.nan, merged[ele].median())

In [ ]:
merged.went_on_backorder.value_counts()/merged.shape[0] * 100

In [ ]:
# Encoding target variable to 0, 1

encode = {
    "No": 0,
    "Yes": 1
}

merged['went_on_backorder'] = merged.went_on_backorder.map(encode)

In [ ]:
merged.head()

### Looking if target is balanced or not ?

In [ ]:
plt.figure(figsize=(7, 5))
sns.countplot(x='went_on_backorder', data=merged)

#### Handling class imbalance

In [ ]:
from imblearn.over_sampling import SMOTE

In [ ]:
X = merged.drop(["sku","went_on_backorder"], axis=1)
y = merged.went_on_backorder

oversampler = SMOTE(random_state=0)
X_res, Y_res = oversampler.fit_resample(X, y)

Y_res.value_counts().plot(kind='bar')   


In [23]:
target_val_check = merged.went_on_backorder.value_counts().to_frame().reset_index().rename(columns={"index": "categories", 
                                                                                 "went_on_backorder": "Counts"})

target_val_check["pct"] = (target_val_check["Counts"]/ len(merged))*100
target_val_check


,categories,Counts,pct
0,No,1915954,99.275469
1,Yes,13981,0.724428


In [26]:
diff_check = target_val_check.iloc[0, 2] - target_val_check.iloc[1, 2]

if diff_check > 20:
    print("Your data is highly imbalance")
else:
    print("Data is balanced")


Your data is highly imbalance


In [ ]:
balanced_dataset = pd.concat([X_res, Y_res], axis=1)